In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Basics of combinatorial circuits using Chisel3 

In this notebook you will get introduced to the basics of Chisel. This will include 
- Modules
 - IOs  
- Chisel types/literals and operators
 - UInt, SInt, Bool.
 - +, -, === etc.
- Wires
- Registers
- Multiplexers



## Module and IO
The first Chisel keyword to be introduced is *Module* and *IO*.  
When declaring a class extending Module, it means that this code will be mapped to a Verilog module. Verilog modules are equivalent to components in VHDL. You start a Module description by declaring the inputs and outputs of the module. This is done using the special keyword *IO*. Declaring IO is equivalent to declaring the entity of a component in VHDL. All signals declared in IO must be specified as either an input or an output, this is done by wrapping the signals in the Input() and Output() apply methods (see code below). The IO declared in the empty module *HardwareModule* has signals from the three basic chisel data types; UInt - unsigned integer, SInt - signed integer and Bool - boolean. You specify the with of the signal bus explicitly using the X.W with x being an integer larger than 0. You cannot set the with of a Bool as it represents a single wire. If you need a bus and don't interpret it as an integer use UInt anyway. Inspect the code below. 

In [ ]:
class HardwareModule extends Module {   // Declare a class extending Module to declare the equivilent of a VHDL component
    val io = IO(new Bundle {            // Declare in and outputs of the module. Equivilent to an entity declaration in VHDL - by convention named "io"
        val input1 = Input(UInt(8.W))   // Declaration of input as a 8-bit bus that can be interpredted as a unsigned integer.
        val input2 = Input(SInt(16.W))  // Declaration of input as a 16-bit bus that can be interpredted as a signed integer.
        
        val output1 = Output(UInt(8.W)) // Declaration of output as a 8-bit bus that can be interpredted as a unsigned integer.
        val output2 = Output(Bool())    // Declaration of output signal. Bool is a signle wire that can either be 0 or 1.
    })
}

### Body of modules
In the body of the module, we describe the circuit of the module. This is equivalent to the architecture declaration in VHDL. We introduce this by making a module that takes three inputs a, b and c and produces the output z = (a + b) * c. This will be described in two steps, to introduce declaration of wires internal to a module, and signal value assignment. wires are declared using Wire(). See code below and read the comments. 

In [ ]:
class AddMult extends Module {
    val io = IO(new Bundle {
        val a = Input(SInt(32.W))
        val b = Input(SInt(32.W))
        val c = Input(SInt(32.W))
        
        val z = Output(SInt(32.W))
    })
    
    val sum = Wire(SInt(32.W))     
    // sum is declared as an internal wire of the SInt type, is a bus that is 32-bits wide.
    val product = Wire(SInt(32.W)) 
    
    sum := io.a + io.b 
    // The line above is an assignment "sum" with the addition of inputs a and b. Assignments are 
    // written with ":=" notice that the inputs are referred to by the prefix "io." which correspond 
    // to the name of the IO of this module. Remember that we are describing hardware, which means 
    // every assignment is evaluated concurrently. This means that you can only assign one value to a
    // wire, otherwise you are describing a short circuit.
    product := sum * io.c 
    io.z := product
    
}
// the following line prints the verilog code that this module generates. This will might give the reader a better
// understanding of whats going on.
println(getVerilog(new AddMult))

## Operaters
Operators such as + and * looks identical for scala and chisel. They are interpreded as hardware if the operand are Chisel types, which is the case for the operators used in the code above. All operands is an assignment must be Chisel types. If you need a satatic value as operand for an operator a postfix is used:

```scala
42.U    // .U specifies the Chisel type UInt
42.S    // .S specifies the Chisel type SInt
true.B  // .B specifies the Chisel type Bool

val a = 42 + 42     // OK - Scala Addition
val b = 42.U + 42.U // OK - Chisel adder unit of adding two static signals.
val c = 42 + 42.U   // Not Cool! - You can't mix scala and chisel types in arithmetic expressions
```

For a list of all Chisel operators please see the official [Chisel3 Cheatsheet](https://www.chisel-lang.org/doc/chisel-cheatsheet3.pdf/)


## Multiplexers
A simple 2:1 multiplexer can descibed using using the Chisel method y := Mux(sel, a, b). here y = a if sel = true.B y = b otherwise. For larger multiplexers multiple 2:1 multiplexers can be used, this is not very elegant instead larger multiplexers are inferred by the use of *switch* or *when* statements. The code below shows how to use each of these methods:

In [ ]:
class AddMult extends Module {
    val io = IO(new Bundle {
        val sel1 = Input(Bool())     // Single wire input to be used as selector for 2:1 Multiplexer
        val sel2 = Input(UInt(2.W))  // 2-bit bus to be used as selector signal for 4:1 multiplexer
        
        val out1 = Output(UInt(8.W)) // Output for 2:1 Mux
        val out2 = Output(UInt(8.W)) // Output for first 4:1 Mux method
        val out3 = Output(UInt(8.W)) // Output for second 2:1 Mux method
    })
    
    io.out1 := Mux(io.sel1, 42.U, 24.U) // 2:1 multiplexer setting output out1 using input sel1 as selector signal
                                        // when input sel1 is true.B out1 is 42.U 24.U otherwise
    
    
    switch(io.sel2){          // This line specify what signal to compare with. Which can be selector signal 
                              // when descripting multiplexer
        is("b00".U) {         // The logic in curly bracket evaluated if io.sel2 === "b00".U.
            io.out2 := 3.U    
        }                     // "b00".U is is equivilant to writing 0.U. This can be nice you are not
        is("b01".U){          // interpreding a UInt bus a an integer. You can write the bit string instead of the
            io.out2 := 23.U   // equivilent integer
        }
        is("b10".U){
            io.out2 := 42.U
        }
        is("b11".U){
            io.out2 := 65.U
        }
    }
 
    
    
//     2   4   5   8
//     |   |   |   |
//     |   |   |   |
//     v   v   v   v
//   XXXXXXXXXXXXXXXXXX
//    XX   4:1 Mux  XX
//      XX        XX<----+sel2
//        XXXXXXXX
//            |
//    2       |
//    |       |
//    v       v
// +--+-------+-+
// | Multiplier |
// +------------+
//       |
//       |    
//       v
//      out3
    
// in the following code a multiplexer is used in the circuit drawn above
    
    val multiplier = Wire(UInt(4.W))
    io.out3 := 2.U * multiplier
    
    when(sel2 === 0.U){          // The logic in the curly bracket are evaluated if the expession in the 
        multiplier := 2.U        // parentheses is true.B
    } .elsewhen(sel2 === 1.U){   // If earlier when statements is false.B and this is true.B this this curly
        multiplier := 4.U        // bracket is evaluated
    } .elsewhen(sel2 === 2.U){
        multiplier := 5.U
    } .otherwise{                // if none of the above when statements is true.B this curky bracket is evaluated.
        multiplier := 7.U
    }
    
}

You now have the tools to build any combinatorial circuit with Chisel. Chisel have a lot methods that makes implementation of complex circuits more elegant. Consult [Chisel3 Cheatsheet](https://www.chisel-lang.org/doc/chisel-cheatsheet3.pdf/) or the book [Digital Design with Chisel](https://github.com/schoeberl/chisel-book/wiki/chisel-book.pdf) to learn more about these.

# Exercise:

Now its your time to try describing a combinatorial circuit using chisel.  

Your task is to implement the circuit drawn below. The circuit are to be implemented in the body of the module "MACorACM" below. The IO for the module have been declared for you. The IO should not be altered as these are used for the Chisel test in the code block following under the module template code.

![CompExImg](images/MACorACM.png)

In [ ]:
class MACorACM extends Module {
    val io = IO(new Bundle {
        val a = Input(UInt(8.W))
        val b = Input(UInt(8.W))
        val c = Input(UInt(8.W))
        val sel = Input(Bool())
      
        val z  = Output(UInt(16.W))
    })
    
    // describe circuit here
}

In the code block below you can test if your implementation is functionally correct. By running the code-block below the chisel PeekPokeTester will test your circuit. if "SUCCESS!!" is printet your circuit description is correct, if not you must fix your description and run another test. Remember to run the code block holding the MACorACM module before testing.
 
 
### Introducing PeekPokeTester

Chisels PeekPokeTester is a tool for during quick test, and to some extend debug of, your circuits description. 
PeekPokeTesters are writtem as classes extending PeekPokeTester. The class must have the module that should be tested as parameter. In the body of the tester class you write a test sequence using four primary Chisel methods:  
- poke(signal, value) - Sets *signal* to *value*. Value doesnt have to be Chisel type, it will be converted automatically.
- step(x) - Sdvances the simulation/test by *x* clock cycles. (Not strickly needed for this test as there is no sequenial components in circuit)
- peek(signal) - Returns the value of the *signal*. Can be printed with *println(peek(signal).toString)*
- expect(signal, value) - Throws an error of *signal* does not hold the *value*.

You when the test has been written it must be executed. This can be done using the Chisel Driver. This can be, done in multipe ways. For now the simplest form is sufficent. The test is run with the command:  
```scala    
Driver(() => new ModuleToTest) {c => new PeekPokeTesterModule(c)}
```
ModuleToTest is the module that you want to test. PeekPokeTesterClass is the PeekPokeTester class descriping the test. c passes the module you want to test as paramter to the PeekPokeTester. (Another name than "c" can be chosen)

In [ ]:
class MACorACMTester(c: MACorACM) extends PeekPokeTester(c) {
  // The parameter c refers to the module we are testing. To access signals from MACorACM 
  // the prefix "c." is therefore needed.
  val tests = 50
  import scala.util.Random
  
    poke(c.io.sel, true)              // Set the selector signal to 1/true.B
  for (i <- 0 until tests) {          // For loop to make 50 tests
    val in_a = Random.nextInt(16)     // Sets the scala values in_a, in_b and in_c to a random integer value 
    val in_b = Random.nextInt(16)     // Between 0 and 16, 16 not included. This range is chosen to avoid overflow.
    val in_c = Random.nextInt(16)
    poke(c.io.a, in_a)                // Sets in MACorACM inputs a, b and c to the random integer value.
    poke(c.io.b, in_b)
    poke(c.io.c, in_c)
    expect(c.io.z, (in_a*in_b)+in_c)  // Tests if the module under test computes the output correctly. If not an 
                                      // an error is thrown
    step(1)                           // Advance the simlation by one clock cycle. Not needed for this test.
  }
    
  poke(c.io.sel, false)               // This loop test the other operation the MACorACM module should compute 
  for (i <- 0 until tests) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    poke(c.io.a, in_a)
    poke(c.io.b, in_b)
    poke(c.io.c, in_c)
    expect(c.io.z, ((in_a+in_b)*in_c))
    step(1)
  }
}
assert(Driver(() => new MACorACM) {c => new MACorACMTester(c)}) // by using assert the next line will not be run 
println("SUCCESS!!")                                            // if all expect statement are not succesful